# Interview q
Given

Happy : [10,152,148,176]

Sad : [10,152,148,176]

Angry : [10,152,148,176]

Surprised : [10,152,148,176]

Neutral : [10,152,148,176]

10 : The left eyebrow in inner corner (also known as the medial corner)

152 : The right eyebrow inner corner

148 : The nose tip

176 : the mouth left corner

Left eyebrow's inner corner(10) = x,y,z ==> LEFT SIDE OF FACE, SLIGHT ABOVE MIDPOINTS, SLIGHTLY FRONT FACE

Eg : x = 0.02, y = 0.1, z = 0.5

In [26]:
import mediapipe as mp
import cv2
import numpy as np

In [27]:
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [28]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

In [29]:
face_mesh = mp_face_mesh.FaceMesh()

In [30]:
# Facial landmarks for expressions
landmark_ids = [10, 152, 148, 176]

In [ ]:
video = cv2.VideoCapture(0)
while True:
    suc, img = video.read()
    img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    result = face_mesh.process(img1)
    
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            mp_drawing.draw_landmarks(img,face_landmarks,mp_face_mesh.FACEMESH_TESSELATION,
                                      mp_drawing.DrawingSpec(color=(0,0,255),thickness=1,circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(255,0,0),thickness=1))
            

            # Extract landmark positions
            h, w, _ = img.shape
            points = {}
            for lm_id in landmark_ids:
                landmark = face_landmarks.landmark[lm_id]
                points[lm_id] = (int(landmark.x * w), int(landmark.y * h))
            
            if len(points) == 4:
                eyebrow_dist = calculate_distance(points[10], points[152])
                nose_mouth_dist = calculate_distance(points[148], points[176])
                
                print(eyebrow_dist, nose_mouth_dist)
                
                # Determine expression
                if 10 <= eyebrow_dist >= 336 and nose_mouth_dist > 20:
                    expression = "Surprised"
                elif 107 > eyebrow_dist < 336 and nose_mouth_dist < 10:
                    expression = "Angry"
                elif 107 <= eyebrow_dist <= 336 and 14 <= nose_mouth_dist <= 17:
                    expression = "Happy"
                elif 57 < eyebrow_dist < 380 and nose_mouth_dist > 12:
                    expression = "Sad"
                else:
                    expression = "Neutral"
                
                cv2.putText(img, expression, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
            
    cv2.imshow("Image",img)

    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

195.12560057562922 16.1245154965971
191.21192431435858 15.524174696260024
188.09572031282372 15.524174696260024
184.0244548966251 14.317821063276353
185.00270268296083 14.317821063276353
190.09471323527123 14.317821063276353
190.06577808748213 14.317821063276353
191.04188022525324 14.317821063276353
190.04210059878838 13.341664064126334
194.0103090044444 14.317821063276353
195.0640920313116 14.317821063276353
193.04144632694815 13.341664064126334
198.00252523642217 13.341664064126334
196.01020381602586 12.36931687685298
197.0101520226813 13.341664064126334
200.0024999843752 12.36931687685298
200.0024999843752 12.649110640673518
202.00990074746338 13.601470508735444
204.00245096566854 12.649110640673518
205.00975586542216 12.36931687685298
206.0218435020908 12.649110640673518
206.0218435020908 12.649110640673518
208.00961516237658 12.36931687685298
209.02152999152983 12.649110640673518
207.02173798903343 12.649110640673518
209.03827400741713 12.36931687685298
209.03827400741713 12.36931

In [53]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Function to calculate Euclidean distance manually
def euclidean_distance(pt1, pt2):
    return ((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2) ** 0.5  # sqrt((x2 - x1)^2 + (y2 - y1)^2)

# Start webcam
video = cv2.VideoCapture(0)

while True:
    ret, img = video.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = img.shape
            # Get key facial landmarks
            points = [face_landmarks.landmark[i] for i in [10, 152, 148, 176]]
            points = [(int(p.x * w), int(p.y * h)) for p in points]

            # Calculate distances manually
            eyebrow_dist = euclidean_distance(points[0], points[1])
            mouth_dist = euclidean_distance(points[2], points[3])

            # Normalize using face width
            face_width = euclidean_distance(points[0], points[3])
            eyebrow_dist /= face_width
            mouth_dist /= face_width

            # Expression classification
            if eyebrow_dist > 0.6 and mouth_dist > 0.15:
                expression = "Surprised"
            elif eyebrow_dist < 0.45 and mouth_dist < 0.10:
                expression = "Angry"
            elif 0.45 <= eyebrow_dist <= 0.55 and 0.10 <= mouth_dist <= 0.12:
                expression = "Happy"
            elif eyebrow_dist < 0.40 and mouth_dist > 0.11:
                expression = "Sad"
            else:
                expression = "Neutral"

            # Display result
            cv2.putText(img, expression, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Face Expression Detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
